## Facial Emotion Recognition:  Classifying Expressions Using Deep Neural Networks

Brief intoriodtion of this projetc

### Install the libraries
The first step is to install all the libraries necessaries.

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

### Check the images
Then we need to dowload the "archieve" folder. This folder need to be download or from the [Kaggle community](https://www.kaggle.com/datasets/msambare/fer2013?select=train), otherwise directly from the GitHub 

In [16]:
img_array = cv2.imread()

SyntaxError: invalid syntax (2365936835.py, line 1)